In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions

es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)


In [2]:
ind="jobs_archive_2016-02-10,jobs_archive_2016-02-11,jobs_archive_2016-02-12,jobs_archive_2016-02-13"

ss = {
  "size": 0,
  "aggs": {
    "2": {
      "terms": {
        "field": "transformation",
        "size": 5,
        "order": {
          "_count": "desc"
        }
      },
      "aggs": {
        "3": {
          "avg": {
            "field": "wall_time"
          }
        },
        "4": {
          "avg": {
            "field": "nevents"
          }
        },
        "5": {
          "avg": {
            "field": "cpu_eff"
          }
        },
        "6": {
          "avg": {
            "field": "corecount"
          }
        },
        "7": {
          "avg": {
            "field": "inputfilebytes"
          }
        },
        "8": {
          "sum": {
            "field": "inputfilebytes"
          }
        }
      }
    }
  },
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False,
          "query": "reqid:5776 OR reqid:5780"
        }
      }
    }
  }
}

In [3]:
res= es.search(index=ind, body=ss, size=3)

#print(res)
for r in res['aggregations']['2']['buckets']:
    cores=r['6']['value']
    wall=r['3']['value']
    events=r['4']['value']
    cpueff=r['5']['value']
    if events<1: continue
    secPerEvent=wall/events*cores
    #print (r['key'], "\tjobs:", r['doc_count'], "\tavg. wall:", wall, "\tevents:", events ,"\tcpu eff.:", cpueff ) 
    print (r['key'], "\tjobs:", r['doc_count'], "\t", secPerEvent, " seconds per event" ) 


Reco_tf.py 	jobs: 200139 	 27.591183340112444  seconds per event
AODMerge_tf.py 	jobs: 9731 	 0.3498153762289544  seconds per event
HISTMerge_tf.py 	jobs: 771 	 0.028787254855824194  seconds per event
http://pandaserver.cern.ch:25085/trf/user/runGen-00-00-02 	jobs: 11 	 0.025362362964568685  seconds per event
